In [4]:
#get the data from the site and unzip the contents to battery_data
!wget -cq https://ti.arc.nasa.gov/c/5 -O naza.zip
!unzip -qqo /Users/jaypalamand/Senior_Design/Frontend/naza.zip -d battery_data

[/Users/jaypalamand/Senior_Design/Frontend/naza.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /Users/jaypalamand/Senior_Design/Frontend/naza.zip or
        /Users/jaypalamand/Senior_Design/Frontend/naza.zip.zip, and cannot find /Users/jaypalamand/Senior_Design/Frontend/naza.zip.ZIP, period.


In [5]:
import datetime
import pandas as pd
from scipy.io import loadmat
from pandas import DataFrame

In [6]:
print("Input which battery you need to extract data from. Choose from the following")
print("Battery Number: B0005,B0006,B0007,B0018")
B = input()

Input which battery you need to extract data from. Choose from the following
Battery Number: B0005,B0006,B0007,B0018


In [71]:
#define a function for extracting discharge and charge data
def disch_data(battery):
  battery = 'B0018'
  mat = loadmat('/Users/jaypalamand/Senior_Design/Frontend/Datasets/mat/B0018.mat') #get the .mat file
  print('Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0])) #get the length of the data from number of cycles
  c = 0 #set a variable to zero
  disdataset = [] #create an empty list for discharge data
  capacity_data = []
  
  for i in range(len(mat[battery][0, 0]['cycle'][0])):
    row = mat[battery][0, 0]['cycle'][0, i] #get each row of the cycle
    if row['type'][0] == 'discharge': #if the row is a dicharge cycle
      ambient_temperature = row['ambient_temperature'][0][0] #get temp,date_time stamp,capacity,voltage,current etc,.
      date_time = datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
      data = row['data']
      capacity = data[0][0]['Capacity'][0][0]
      for j in range(len(data[0][0]['Voltage_measured'][0])):
        voltage_measured = data[0][0]['Voltage_measured'][0][j]
        current_measured = data[0][0]['Current_measured'][0][j]
        temperature_measured = data[0][0]['Temperature_measured'][0][j]
        current_load = data[0][0]['Current_load'][0][j]
        voltage_load = data[0][0]['Voltage_load'][0][j]
        time = data[0][0]['Time'][0][j]
        disdataset.append([c + 1, ambient_temperature, date_time, capacity,
                        voltage_measured, current_measured,
                        temperature_measured, current_load,
                        voltage_load, time])
        capacity_data.append([c + 1, ambient_temperature, date_time, capacity])
      c = c + 1
  print(disdataset[0])
  return [pd.DataFrame(data=disdataset,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity', 'voltage_measured',
                                'current_measured', 'temperature_measured',
                                'current_load', 'voltage_load', 'time']),
          pd.DataFrame(data=capacity_data,
                       columns=['cycle', 'ambient_temperature', 'datetime',
                                'capacity'])]

def charge_data(battery): #similarly write a fn for charge data
  mat = loadmat('/Users/jaypalamand/Senior_Design/Frontend/Datasets/mat/B0018.mat')
  c = 0
  chdataset = []
  battery = 'B0018'
  for i in range(len(mat[battery][0, 0]['cycle'][0])):
    row = mat[battery][0, 0]['cycle'][0, i]
    if row['type'][0] == 'charge' :
            
      ambient_temperature = row['ambient_temperature'][0][0]
      date_time = datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
      data = row['data']
      for j in range(len(data[0][0]['Voltage_measured'][0])):
        voltage_measured = data[0][0]['Voltage_measured'][0][j]
        current_measured = data[0][0]['Current_measured'][0][j]
        temperature_measured = data[0][0]['Temperature_measured'][0][j]
        current_charge = data[0][0]['Current_charge'][0][j]
        voltage_charge = data[0][0]['Voltage_charge'][0][j]
        time = data[0][0]['Time'][0][j]
        chdataset.append([c + 1, ambient_temperature, date_time,
                        voltage_measured, current_measured,
                        temperature_measured, current_charge,
                        voltage_charge, time])
      c = c + 1
  print(chdataset[788])
  return chdataset

def impedance_data(battery):
    mat = loadmat(f'/Users/jaypalamand/Senior_Design/Frontend/Datasets/mat/B0018.mat')
    battery = 'B0018'
    c = 0
    impdataset = []
    
    for i in range(len(mat[battery][0, 0]['cycle'][0])):
        row = mat[battery][0, 0]['cycle'][0, i]
        if row['type'][0] == 'impedance':
            ambient_temperature = row['ambient_temperature'][0][0]
            date_time = datetime.datetime(int(row['time'][0][0]),
                                          int(row['time'][0][1]),
                                          int(row['time'][0][2]),
                                          int(row['time'][0][3]),
                                          int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
            data = row['data']
            for j in range(len(data[0][0]['Battery_impedance'][0])):
                print(data[0][0]['Sense_current'])
                sense_current = data[0][0]['Sense_current'][0][j]
                battery_current = data[0][0]['Battery_current'][0][j]
                battery_impedance = data[0][0]['Battery_impedance'][0][j]
                rectified_impedance = data[0][0]['Rectified_Impedance'][0][j]
                re = data[0][0]['Re'][0][j]
                rct = data[0][0]['Rct'][0][j]
                impdataset.append([c + 1, ambient_temperature, date_time,
                                   sense_current, battery_current, battery_impedance,
                                   rectified_impedance, re, rct])
            c += 1

    return pd.DataFrame(data=impdataset,
                        columns=['cycle', 'ambient_temperature', 'datetime',
                                 'sense_current', 'battery_current', 'battery_impedance',
                                 'rectified_impedance', 'Re', 'Rct'])

In [72]:
chdataset = charge_data(B)
chdf=pd.DataFrame(data=chdataset,columns=['cycle', 'ambient_temperature', 'datetime', 
                                'voltage_measured','current_measured',
                                'temperature_measured','current',
                                'voltage', 'time'])
pd.set_option('display.max_columns', 10)
chdf.to_csv('/Users/jaypalamand/Senior_Design/csv_data/B0018_charge.csv')

[1, 24, datetime.datetime(2008, 7, 7, 12, 26, 45), 4.199114407695108, 0.3979857346270783, 24.824364773359807, 0.398, 4.397, 2207.031]


In [73]:
dataset = impedance_data('B0006')
dataset.to_csv('/Users/jaypalamand/Senior_Design/csv_data/B0018_impedance.csv')

[[832.74957275-2.73771935e+01j 826.06286621-4.12816010e+01j
  825.53729248-4.86207275e+01j 823.86834717-5.54931030e+01j
  823.28155518-5.35582809e+01j 823.10015869-5.75157166e+01j
  821.27819824-6.28570709e+01j 821.62835693-6.34944687e+01j
  818.85656738-6.76813507e+01j 818.45910645-6.62869568e+01j
  819.05712891-5.94921646e+01j 820.42468262-5.94221039e+01j
  818.87695312-5.92895737e+01j 817.42572021-6.10955505e+01j
  814.84179688-6.05709534e+01j 817.37756348-3.72448845e+01j
  815.31323242-4.17742996e+01j 814.50762939-4.01737289e+01j
  812.71893311-3.92747612e+01j 813.13885498-4.09638290e+01j
  811.06427002-4.20640640e+01j 810.60125732+3.71559143e-01j
  807.41723633+7.50356674e-01j 806.06109619+3.83506775e-01j
  805.56860352+6.71564102e-01j 804.8192749 +7.14544296e-01j
  799.57855225-1.37956238e+00j 801.14923096-9.10913467e+00j
  798.55212402-1.19346256e+01j 804.61175537-1.72334156e+01j
  798.00506592-1.51957169e+01j 802.8729248 -1.87379303e+01j
  796.40942383-3.40101318e+01j 797.48785

In [36]:
def print_field_names(battery, cycle_type):
    mat = loadmat('/Users/jaypalamand/Senior_Design/Frontend/Datasets/mat/B0005.mat')
    cycle_data = mat[battery][0, 0]['cycle'][0]
    for cycle in cycle_data:
        if cycle['type'][0] == cycle_type:
            print(f"Fields for cycle type '{cycle_type}':")
            for field in cycle['data'].dtype.names:
                print(field)
            break  # Break after printing fields for the first matching cycle

# Example usage:
print_field_names('B0005', 'impedance')

Fields for cycle type 'impedance':
Sense_current
Battery_current
Current_ratio
Battery_impedance
Rectified_Impedance
Re
Rct


In [74]:
disdf,capacity = disch_data(B)
pd.set_option('display.max_columns', 10)
disdf.to_csv('/Users/jaypalamand/Senior_Design/csv_data/B0018_discharge.csv')

Total data in dataset:  319
[1, 24, datetime.datetime(2008, 7, 7, 15, 15, 28), 1.8550045207910817, 4.188108651124536, 0.00013066734156636677, 23.8195202516044, 0.0006, 0.0, 0.0]


In [13]:
capacity

,cycle,ambient_temperature,datetime,capacity
0,1,4,2010-09-03 12:10:27,0.785278
1,1,4,2010-09-03 12:10:27,0.785278
2,1,4,2010-09-03 12:10:27,0.785278
3,1,4,2010-09-03 12:10:27,0.785278
4,1,4,2010-09-03 12:10:27,0.785278
...,...,...,...,...
22657,102,4,2010-09-30 11:50:17,1.129059
22658,102,4,2010-09-30 11:50:17,1.129059
22659,102,4,2010-09-30 11:50:17,1.129059
22660,102,4,2010-09-30 11:50:17,1.129059


In [48]:
import sqlite3
import pandas as pd
import sqlalchemy

In [55]:
engine = sqlalchemy.create_engine('sqlite:///Battery.db')
del engine

In [51]:
import sqlite3
connection = sqlite3.connect('Battery.db')

In [52]:
cursor = connection.cursor()

In [75]:
charge_df1 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0005_charge.csv')
discharge_df1 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0005_discharge.csv')
impedance_df1 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0005_impedance.csv')
charge_df2 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0006_charge.csv')
discharge_df2 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0006_discharge.csv')
impedance_df2 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0006_impedance.csv')
charge_df3 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0007_charge.csv')
discharge_df3 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0007_discharge.csv')
impedance_df3 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0007_impedance.csv')
charge_df4 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0018_charge.csv')
discharge_df4 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0018_discharge.csv')
impedance_df4 = pd.read_csv('/Users/jaypalamand/Senior_Design/csv_data/B0018_impedance.csv')


In [76]:
import pandas as pd
from sqlalchemy import create_engine

# Initialize a database connection
# 'sqlite:///battery_data.db' specifies a new SQLite database named 'battery_data.db'
engine = create_engine('sqlite:///battery_data.db')
connection = engine.raw_connection()


# Assuming you have a dictionary of batteries where each key is a battery ID and each value is a dictionary of DataFrames
batteries = {
    'B0005': {
        'charge': charge_df1,
        'discharge': discharge_df1,
        'impedance': impedance_df1
    },
    'B0006': {
        'charge': charge_df2,
        'discharge': discharge_df2,
        'impedance': impedance_df2
    },
    'B0007': {
        'charge': charge_df3,
        'discharge': discharge_df3,
        'impedance': impedance_df3
    },
    'B0018': {
        'charge': charge_df4,
        'discharge': discharge_df4,
        'impedance': impedance_df4
    },
    # Add more batteries as needed
}

for battery_id, tables in batteries.items():
    for table_name, df in tables.items():
        # Construct a table name uniquely identifying each battery and its measurement type
        full_table_name = f"{battery_id}_{table_name}"
        # Write the DataFrame to a SQL table directly using the engine
        df.to_sql(full_table_name, connection, if_exists='replace', index=False)


/var/folders/ns/vlr7gfqx2nq9l1xdf8fs7y480000gn/T/ipykernel_52550/1950833885.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(full_table_name, connection, if_exists='replace', index=False)


In [61]:
import sqlite3

connection = sqlite3.connect('Database.db')
cursor = connection.cursor()

cursor.execute('CREATE TABLE battery(name TEXT, salary REAL)')

In [62]:
connection.commit()

In [64]:
import pyspark